# **Sistemas Inteligentes** - Examen de laboratorio (CNNs)

## Datos personales

In [ ]:
import tensorflow as tf

#@title  { run: "auto", display-mode: "form" }
Nombre = "Juan Francisco" #@param {type:"string"}
Apellidos = "Mier Montoto" #@param {type:"string"}
DNI = "71777658V" #@param {type:"string"}
UO = 283319 #@param {type:"integer"}

if UO is not None:
  tf.random.set_seed(2032)  

## Conjunto de datos

In [ ]:
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
batch_size = 64
num_classes = 100
val_size = 0.1  # proporción 90:10

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

datagen = ImageDataGenerator(featurewise_center=True,
                             featurewise_std_normalization=True,
                             validation_split=val_size)
datagen.fit(x_train)
data_train = datagen.flow(x_train, y_train, batch_size=batch_size, subset="training")
data_val = datagen.flow(x_train, y_train, batch_size=batch_size, subset="validation")
data_test = datagen.flow(x_test, y_test, batch_size=batch_size)

169001437/169001437 [==============================] - 2s 0us/step


## Arquitectura

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Flatten

base_model = ResNet50(weights='imagenet', include_top=True)

for layer in base_model.layers:
  layer.trainable = False  # "Los parámetros no se ajustarán durante la fase de entrenamiento" 

x = base_model.output
x = Flatten()(x) # Transformar el volumen de características extraído por la base en un vector
x = Dense(500)(x)
x = Dense(250)(x)
predictions = Dense(num_classes, activation='softmax')(x) 

model = Model(inputs=[base_model.input], outputs=[predictions])
model.summary()

102967424/102967424 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                           

## Entrenamiento

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='categorical_crossentropy',  # 'categorical_crossentropy' porque es un problema de clasificación multiclase
              optimizer=RMSprop(learning_rate=0.001),  
              metrics=['accuracy'])

model.fit(data_train,
          epochs=2,   
          verbose=1,  
          steps_per_epoch=(1-val_size)*len(y_train)/batch_size,
          validation_data=data_val,
          validation_steps=val_size*len(y_train)/batch_size)

Epoch 1/2
703/703 [==============================] - 31s 23ms/step - loss: 4.4175 - accuracy: 0.0386 - val_loss: 4.2805 - val_accuracy: 0.0530
Epoch 2/2
703/703 [==============================] - 13s 19ms/step - loss: 4.2456 - accuracy: 0.0610 - val_loss: 4.1955 - val_accuracy: 0.0734



## ANEXO

### Conjunto de datos

#### Conjuntos de imágenes disponibles en Tensorflow

*  `tf.keras.datasets.cifar10`: This is a dataset of 50,000 32x32 color training images and 10,000 test images, labeled over 10 categories. 
*  `tf.keras.datasets.cifar100`: This is a dataset of 50,000 32x32 color training images and 10,000 test images, labeled over 100 fine-grained classes.
*  `tf.keras.datasets.fashion_mnist`: This is a dataset of 60,000 28x28 grayscale images of 10 fashion categories, along with a test set of 10,000 images.
*  `tf.keras.datasets.mnist`: This is a dataset of 60,000 28x28 grayscale images of the 10 digits, along with a test set of 10,000 images.

Método para cargar los datos:
*  `load_data()`: Loads the specified dataset.

```python
from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
```

#### Generar *batches* de imágenes

*  `tf.keras.preprocessing.image.ImageDataGenerator(featurewise_center=False, featurewise_std_normalization=False, rotation_range=0, horizontal_flip=False, vertical_flip=False, rescale=None, validation_split=0.0)`: Generates batches of tensor image data with real-time data augmentation.
>  *  `featurewise_center`: Boolean. Set input mean to 0 over the dataset, feature-wise.
>  *  `featurewise_std_normalization`: Boolean. Divide inputs by std of the dataset, feature-wise.
>  *  `rotation_range`: Int. Degree range for random rotations.
>  *  `horizontal_flip`: Boolean. Randomly flip inputs horizontally.
>  *  `vertical_flip`: Boolean. Randomly flip inputs vertically.
>  *  `rescale`: Rescaling factor.
>  *  `validation_split`: Float. Fraction of images reserved for validation (strictly between 0 and 1).

Métodos de `ImageDataGenerator`:

*  `fit(x)`: Fits the data generator to some sample data.
>  *  `x`: Sample data. 

*  `flow(x, y=None, batch_size=32, subset=None)`: Takes data and label arrays, generates batches of augmented data.
>  *  `x`: Input data. 
>  *  `y`: Labels.
>  *  `batch_size`: Int.
>  *  `subset`: Subset of data ('training' or 'validation')

```python
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(featurewise_center=True,
                             featurewise_std_normalization=True,
                             validation_split=0.2)
datagen.fit(x_train)
data_train = datagen.flow(x_train, y_train, batch_size=256, subset="training")
```

#### Otras operaciones

*  `pandas.read_csv(filepath, dtype=None)`: Read a comma-separated values (csv) file into DataFrame.
>  *  `filepath`: Any valid string path.
>  *  `dtype`: Data type for data or columns. 

```python
import pandas as pd
labels = pd.read_csv("labels.csv", dtype = {"class": "category"})
```

*  `tf.keras.utils.to_categorical(y, num_classes)`: Converts a class vector (integers) to binary class matrix.
>  *  `y`: Array-like with class values to be converted into a matrix (integers from 0 to num_classes - 1).
>  *  `num_classes`: Total number of classes. 

```python
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, 5)
```

*  `sklearn.model_selection.train_test_split(arrays, test_size=None, train_size=None)`: Split arrays or matrices into random train and test subsets.
>  *  `arrays`: Allowed inputs are lists, numpy arrays, scipy-sparse matrices or pandas dataframes.
>  *  `test_size`: If float, should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the test split.
>  *  `train_size`: If float, should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the train split.

```python
from sklearn.model_selection import train_test_split
train_data, val_data = train_test_split(data, train_size=0.8)
```


### Arquitectura

#### Capas

*  `tf.keras.layers.Conv2D(filters, kernel_size, strides=(1, 1),padding='valid', activation=None, use_bias=True)`: 2D convolution layer.
>  *  `filters`: Integer, the dimensionality of the output space (i.e. the number of output filters in the convolution).
>  *  `kernel_size`: An integer or tuple/list of 2 integers, specifying the height and width of the 2D convolution window. Can be a single integer to specify the same value for all spatial dimensions.
>  *  `strides`: An integer or tuple/list of 2 integers, specifying the strides of the convolution along the height and width. Can be a single integer to specify the same value for all spatial dimensions.
>  *  `padding`: one of "valid" or "same" (case-insensitive). "valid" means no padding. "same" results in padding with zeros evenly to the left/right or up/down of the input. When padding="same" and strides=1, the output has the same size as the input.
>  *  `activation`: Activation function to use. If you don't specify anything, no activation is applied. Possible values: 'relu', 'sigmoid', 'softmax', etc.
>  *  `use_bias`: Boolean, whether the layer uses a bias vector.

```python
from tensorflow.keras.layers import Conv2D
Conv2D(16, 5, activation='relu', input_shape=(64,64,1))
```

*  `tf.keras.layers.Dense(units, activation=None, use_bias=True)`: Just your regular densely-connected NN layer.
>  *  `units`: Positive integer, dimensionality of the output space.
>  *  `activation`: Activation function to use. If you don't specify anything, no activation is applied. Possible values: 'relu', 'selu', 'sigmoid', 'softmax', 'softplus', 'tanh', etc.
>  *  `use_bias`: Boolean, whether the layer uses a bias vector.

```python
from tensorflow.keras.layers import Dense
Dense(256, activation='relu')
```

*  `tf.keras.layers.Dropout(rate)`: Applies Dropout to the input.
>  *  `rate`: Float between 0 and 1. Fraction of the input units to drop.

```python
from tensorflow.keras.layers import Dropout
Dropout(0.5)
```

*  `tf.keras.layers.Flatten()`: Flattens the input. 
*  `tf.keras.layers.GlobalMaxPool2D()`: Global max pooling operation for spatial data.
*  `tf.keras.layers.GlobalAveragePooling2D()`: Global average pooling operation for spatial data.
*  `tf.keras.layers.MaxPool2D()`: Max pooling operation for 2D spatial data.
*  `tf.keras.layers.AveragePooling2D()`: Average pooling operation for 2D spatial data.







#### Modelo secuencial

*  `tf.keras.Sequential()`: Sequential groups a linear stack of layers into model.

```python
from tensorflow.keras.models import Sequential
model = Sequential()
```

Métodos de `Sequential`:
*  `add(layer)`: Adds a layer instance on top of the layer stack.
>  *  `layer`: layer instance.

```python
from tensorflow.keras.layers import Flattern
model.add(Flatten())
```

*  `compile(optimizer='rmsprop',loss=None,metrics=None)`: Configures the model for training.
>  *  `optimizer`: String (name of optimizer) or optimizer instance. Ver sección "Optimizadores" en el ANEXO.
>  *  `loss`: Loss function. Possible values: 'mean_squared_error', 'binary_crossentropy', 'categorical_crossentropy', etc.
>  *  `metrics`: List of metrics to be evaluated by the model during training and testing. Possible values: 'accuracy', 'mse', etc.

```python
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,  
              metrics=['accuracy'])
```

*  `evaluate(x=None, y=None, batch_size=None, verbose='auto' steps=None)`: Returns the loss value & metrics values for the model in test mode.
>  *  `x`: Input data.
>  *  `y`: Target data. 
>  *  `batch_size`: Integer or None. Number of samples per batch of computation.
>  *  `verbose`: 'auto', 0, 1, or 2. Verbosity mode. 0 = silent, 1 = progress bar, 2 = single line.
>  *  `steps`: Integer or None. Total number of steps (batches of samples) before declaring the evaluation round finished.

```python
loss, acc = model.evaluate(data_test,
                           steps=10,
                           verbose=2)
```

*  `fit(x=None, y=None, batch_size=None, epochs=1, verbose='auto', validation_split=0.0, validation_data=None steps_per_epoch=None, validation_steps=None, validation_batch_size=None)`: Trains the model for a fixed number of epochs (dataset iterations).
>  *  `x`: Input data.
>  *  `y`: Target data. 
>  *  `batch_size`: Integer or None. Number of samples per gradient update.
>  *  `epochs`: Integer. Number of epochs to train the model. 
>  *  `verbose`: 'auto', 0, 1, or 2. Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch.
>  *  `validation_split`: Float between 0 and 1. Fraction of the training data to be used as validation data.
>  *  `validation_data`: Data on which to evaluate the loss and any model metrics at the end of each epoch. The model will not be trained on this data. 
>  *  `steps_per_epoch`: Integer or None. Total number of steps (batches of samples) before declaring one epoch finished and starting the next epoch. 
>  *  `validation_steps`: Total number of steps (batches of samples) to draw before stopping when performing validation at the end of every epoch.
>  *  `validation_batch_size`: Integer or None. Number of samples per validation batch. 

```python
model.fit(data_train,
          epochs=6,   
          verbose=2,  
          steps_per_epoch=20,
          validation_data=data_val,
          validation_steps=10)
```

*  `predict(x, batch_size=None, verbose='auto')`: Generates output predictions for the input samples.
>  *  `x`: Input samples.
>  *  `batch_size`: Integer or None. Number of samples per batch.
>  *  `verbose`: 'auto', 0, 1, or 2. Verbosity mode. 0 = silent, 1 = progress bar, 2 = single line.

```python
import numpy as np
predictions = model.predict(data_test, verbose=2)
print(np.argmax([predictions[0]]), np.max(predictions[0]))
```

*  `summary()`: Prints a string summary of the network.

#### Modelos pre-entrenados en TensorFlow

*  `tf.keras.applications.DenseNet121(include_top=True,weights='imagenet', input_shape=None)`: Instantiates the Densenet121 architecture.
 
*  `tf.keras.applications.EfficientNetB0(include_top=True,weights='imagenet', input_shape=None)`: Instantiates the EfficientNetB0 architecture.

*  `tf.keras.applications.InceptionV3(include_top=True, weights='imagenet', input_shape=None,)`: Instantiates the Inception v3 architecture.

*  `tf.keras.applications.ResNet50(include_top=True, weights='imagenet', input_shape=None,)`: Instantiates the ResNet50 architecture.
>  *  `include_top`: Boolean, whether to include the fully-connected layer at the top, as the last layer of the network.
>  *  `weights`: One of None (random initialization), imagenet (pre-training on ImageNet), or the path to the weights file to be loaded. 
>  *  `input_shape`: Optional shape tuple, only to be specified if include_top is False.

```python
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

base_model = InceptionV3(weights='imagenet', include_top=True)

for layer in model.layers:
  layer.trainable = True

x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(3, activation='softmax')(x) 

model = Model(inputs=[base_model.input], outputs=[predictions])
```


#### Optimizadores

*  `tf.keras.optimizers.Adam(learning_rate=0.001)`: Optimizer that implements the Adam algorithm.

*  `tf.keras.optimizers.experimental.Adadelta(learning_rate=0.001)`: Optimizer that implements the Adadelta algorithm.

*  `tf.keras.optimizers.experimental.RMSprop(learning_rate=0.001)`: Optimizer that implements the RMSprop algorithm.

*  `tf.keras.optimizers.experimental.SGD(learning_rate=0.01)`: Gradient descent (with momentum) optimizer.
>  *  `learning_rate`: The learning rate. 


```python
from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate=0.0001)
```